In [3]:
pip install mysql-connector-python pandas

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install chardet


Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import numpy as np

In [6]:
import chardet

# Detect file encoding
with open("uk_sales_data.csv", "rb") as f:
    result = chardet.detect(f.read(100000))

print("Detected encoding:", result['encoding'])

Detected encoding: ascii


In [7]:
import pandas as pd

# Replace 'ISO-8859-1' with the detected encoding if different
df = pd.read_csv("uk_sales_data.csv", encoding="ISO-8859-1")


In [8]:
df.columns
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12-01-2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12-01-2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12-01-2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12-01-2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12-01-2010 08:26,3.39,17850.0,United Kingdom


In [9]:
# Check basic info
df.info()

# Check for missing values
print(df.isnull().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB
InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64


Data Cleaning

In [11]:
# Description (1,454 missing values):filling them with "Unknown"
df["Description"] = df["Description"].fillna("Unknown")

In [12]:
# CustomerID (135,080 missing values): Since this is a large portion of the data (~25%), filling them with guest:
df["CustomerID"] = df["CustomerID"].fillna("Guest")

In [13]:
#Cleaning invoice before conversion:
df["InvoiceDate"] = df["InvoiceDate"].str.replace("/", "-", regex=True).str.strip()
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], errors="coerce")


In [14]:
# Converting invoice date to datetimeformat
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], format="%m/%d/%Y %H:%M")

In [15]:
#Checking
print(df["InvoiceDate"].isna().sum(), "rows could not be converted.")
df[df["InvoiceDate"].isna()].head()  # View problematic rows


0 rows could not be converted.


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


In [16]:
# Removing negative vales for quantity and unit price
df = df[df["Quantity"] > 0]
df = df[df["UnitPrice"] > 0]


In [17]:
# Removing duplicates
df.drop_duplicates(inplace=True)

In [18]:
# Standardize text columns

df["Description"] = df["Description"].str.strip().str.lower()
df["Country"] = df["Country"].str.strip().str.title()


Feature Engineering

In [20]:
#Total price per transaction:

df["TotalPrice"] = df["Quantity"] * df["UnitPrice"]

In [21]:
#Extract date components:

df["Year"] = df["InvoiceDate"].dt.year
df["Month"] = df["InvoiceDate"].dt.month
df["DayOfWeek"] = df["InvoiceDate"].dt.day_name()

In [22]:
#Cleaned Data
df.info()
df.describe()
df.head()


<class 'pandas.core.frame.DataFrame'>
Index: 524878 entries, 0 to 541908
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    524878 non-null  object        
 1   StockCode    524878 non-null  object        
 2   Description  524878 non-null  object        
 3   Quantity     524878 non-null  int64         
 4   InvoiceDate  524878 non-null  datetime64[ns]
 5   UnitPrice    524878 non-null  float64       
 6   CustomerID   524878 non-null  object        
 7   Country      524878 non-null  object        
 8   TotalPrice   524878 non-null  float64       
 9   Year         524878 non-null  int32         
 10  Month        524878 non-null  int32         
 11  DayOfWeek    524878 non-null  object        
dtypes: datetime64[ns](1), float64(2), int32(2), int64(1), object(6)
memory usage: 48.1+ MB


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice,Year,Month,DayOfWeek
0,536365,85123A,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,2010,12,Wednesday
1,536365,71053,white metal lantern,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010,12,Wednesday
2,536365,84406B,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,2010,12,Wednesday
3,536365,84029G,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010,12,Wednesday
4,536365,84029E,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010,12,Wednesday


In [23]:
df


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice,Year,Month,DayOfWeek
0,536365,85123A,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,2010,12,Wednesday
1,536365,71053,white metal lantern,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010,12,Wednesday
2,536365,84406B,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,2010,12,Wednesday
3,536365,84029G,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010,12,Wednesday
4,536365,84029E,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010,12,Wednesday
...,...,...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,pack of 20 spaceboy napkins,12,2011-12-09 12:50:00,0.85,12680.0,France,10.20,2011,12,Friday
541905,581587,22899,children's apron dolly girl,6,2011-12-09 12:50:00,2.10,12680.0,France,12.60,2011,12,Friday
541906,581587,23254,childrens cutlery dolly girl,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60,2011,12,Friday
541907,581587,23255,childrens cutlery circus parade,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60,2011,12,Friday


In [24]:
# Step 1: DimCustomer
dim_customer = df[['CustomerID', 'Country']].drop_duplicates(subset='CustomerID', keep='first').reset_index(drop=True)

# Step 2: DimProduct
# Keep the first occurrence of each StockCode
dim_product = (
    df[['StockCode', 'Description', 'UnitPrice']]
    .drop_duplicates(subset='StockCode', keep='first')
    .reset_index(drop=True)
)


# Step 3: DimDate
dim_date = df[['InvoiceDate', 'Year', 'Month', 'DayOfWeek']].drop_duplicates(subset='InvoiceDate', keep='first').reset_index(drop=True)

# Optionally create a surrogate key for DimDate
dim_date['DateID'] = dim_date.index + 1  # or use pd.factorize if needed
dim_date = dim_date[['DateID', 'InvoiceDate', 'Year', 'Month', 'DayOfWeek']]

# Step 4: FactSales (Fact Table)
# Join to get DateID from DimDate
fact_sales = df.merge(dim_date, on='InvoiceDate', how='left')
fact_sales = fact_sales[['InvoiceNo', 'StockCode', 'CustomerID', 'Quantity', 'TotalPrice', 'DateID']]

# Reset index
fact_sales = fact_sales.reset_index(drop=True)

# Display shapes as a quick check
print("DimCustomer:", dim_customer.shape)
print("DimProduct:", dim_product.shape)
print("DimDate:", dim_date.shape)
print("FactSales:", fact_sales.shape)

DimCustomer: (4339, 2)
DimProduct: (3922, 3)
DimDate: (18499, 5)
FactSales: (524878, 6)


In [25]:
# Check for duplicate CustomerID values
duplicates = dim_customer[dim_customer.duplicated(subset='CustomerID', keep=False)]

# Show summary
print(f"Total duplicate CustomerIDs: {duplicates['CustomerID'].nunique()}")
print(f"Total duplicate rows: {duplicates.shape[0]}")

# Display the duplicate rows
print(duplicates.sort_values('CustomerID').head())


Total duplicate CustomerIDs: 0
Total duplicate rows: 0
Empty DataFrame
Columns: [CustomerID, Country]
Index: []


In [26]:
dim_product

,StockCode,Description,UnitPrice
0,85123A,white hanging heart t-light holder,2.55
1,71053,white metal lantern,3.39
2,84406B,cream cupid hearts coat hanger,2.75
3,84029G,knitted union flag hot water bottle,3.39
4,84029E,red woolly hottie white heart.,3.39
...,...,...,...
3917,85179a,green bitty light chain,2.46
3918,23617,set 10 cards swirly xmas tree 17104,2.91
3919,90214U,"letter ""u"" bling key ring",0.29
3920,47591b,scotties childrens apron,4.13


In [27]:
df.to_excel("cleaned_data_uk_excel.xlsx", index=False)

In [28]:
import pandas as pd

df = pd.read_csv("cleaned_data_uk.csv", dtype={"InvoiceNo": str}, low_memory=False)
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])  # Convert InvoiceDate column


In [29]:
import pymysql
import pandas as pd

# Load the CSV file
df = pd.read_csv("cleaned_data_uk.csv", dtype={"InvoiceNo": str}, low_memory=False)
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

# Connect to MySQL
conn = pymysql.connect(
    host="localhost",
    user="root",
    password="pass",  # Your MySQL password
    database="verodat"
)
cursor = conn.cursor()

# Insert data into MySQL table
for _, row in df.iterrows():
    sql = """
    INSERT INTO ecommerce_sales (InvoiceNo, StockCode, Description, Quantity, InvoiceDate, UnitPrice, CustomerID, Country, TotalPrice, Year, Month, DayOfWeek)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    values = (
        row["InvoiceNo"], row["StockCode"], row["Description"], row["Quantity"], row["InvoiceDate"], 
        row["UnitPrice"], row["CustomerID"], row["Country"], row["TotalPrice"], row["Year"], row["Month"], row["DayOfWeek"]
    )
    cursor.execute(sql, values)

# Commit and close connection
conn.commit()
cursor.close()
conn.close()

print("Data uploaded successfully!")


Data uploaded successfully!


In [30]:
import pandas as pd

# Load the dataset
df = pd.read_csv("fact_sales.csv")

C:\Users\Venkat Naveen\AppData\Local\Temp\ipykernel_25416\120078642.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("fact_sales.csv")


In [31]:
df.shape

(524878, 6)

In [32]:
# Show shape before deduplication
print(f"Original shape: {df.shape}")

Original shape: (524878, 6)


In [33]:
# Drop exact duplicate rows (all columns match)
df_cleaned = df.drop_duplicates().reset_index(drop=True)

# Show shape after deduplication
print(f"After removing exact duplicates: {df_cleaned.shape}")

After removing exact duplicates: (524876, 6)


In [34]:
# Step 1: Remove rows where CustomerID is 'Guest' since it is a primary key
df = df[df['CustomerID'] != 'Guest']

In [35]:
# Step 2: Convert CustomerID to integer (after removing 'Guest')
df['CustomerID'] = df['CustomerID'].astype(float).astype(int)

# Optional: Reset index
df = df.reset_index(drop=True)

In [36]:
df

,InvoiceNo,StockCode,CustomerID,Quantity,TotalPrice,DateID
0,536365,85123A,17850,6,15.30,1
1,536365,71053,17850,6,20.34,1
2,536365,84406B,17850,8,22.00,1
3,536365,84029G,17850,6,20.34,1
4,536365,84029E,17850,6,20.34,1
...,...,...,...,...,...,...
392687,581587,22613,12680,12,10.20,18499
392688,581587,22899,12680,6,12.60,18499
392689,581587,23254,12680,4,16.60,18499
392690,581587,23255,12680,4,16.60,18499


In [37]:
df.to_csv("C:/Users/Venkat Naveen/Desktop/verodat uk sales/fact_sales.csv", index=False)

In [38]:
# Check for nulls
print(dim_date['DateID'].isnull().sum())

# Check for unmatched values
unmatched = dim_date[~dim_date['DateID'].isin(dim_date['DateID'])]
print(f"Unmatched DateIDs: {unmatched['DateID'].nunique()}")


0
Unmatched DateIDs: 0


In [39]:
dim_customer.to_csv("C:/Users/Venkat Naveen/Desktop/verodat uk sales/dim_customer.csv", index=False)
dim_product.to_csv("C:/Users/Venkat Naveen/Desktop/verodat uk sales/dim_product.csv", index=False)
dim_date.to_csv("C:/Users/Venkat Naveen/Desktop/verodat uk sales/dim_date.csv", index=False)
fact_sales.to_csv("C:/Users/Venkat Naveen/Desktop/verodat uk sales/fact_sales.csv", index=False)